# Section 3.2 LightGBM with Bayesian Optimisation

In [ ]:
# 把Google Drive挂载到Colab里
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())
get_root_dir()

'/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks'

In [ ]:
!ls

'1.1 EDA.ipynb'
'1.2.1 Fill missing values by RF (train set).ipynb'
'1.2.2 Fill missing values by RF (test set).ipynb'
'1.3 Data Processing Demo.ipynb'
'2.3.1 Feature Engineering-1.ipynb'
'2.3.2 Feature Engineering-2.ipynb'
'2.3.3 Feature Engineering-3.ipynb'
'2.3.4 Feature Engineering-4.ipynb'
'2.3.5 Feature Engineering-5.ipynb'
'2.3.6 Feature Engineering-6.ipynb'
'2.3.7 Feature Engineering-7.ipynb'
'2.3.8 Feature Engineering-8.ipynb'
'2.4.1 Interaction Feature - Split Test & Train.ipynb'
'2.4.2 Interaction Feature Selection - Filter Method.ipynb'
'2.5 Wrapper Method.ipynb'
'2.6 Combining Part 1 and 2 & Normalizing.ipynb'
'3.2 LightGBM with Bayesian Optimisation.ipynb'
'3.2 LightGBM with Bayesian Optimisation (Try to run on GPU).ipynb'
'3.3 XGBoost - Training.ipynb'
'3.6.1 MLP Bagging Training.ipynb'
'3.6.2 MLP Bagging Inference.ipynb'
'4.1 LSTM Filling Missing Records.ipynb'
'4.2 amex-lstm-pytorch-lightning-training-cleaning-features.ipynb'
'4.3 amex-lstm-pytorch-lightning-inf-featur

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM -b v3.1.1

fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Found Boost: /usr/include (found suitable version "1.65.1", minimum required is "1.56.0") found components: filesystem system 
-- Performing Test MM_PREFET

In [ ]:
!pip install lightgbm==3.1.1
import lightgbm as lgb
lgb.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'3.1.1'

In [ ]:
!pip install hyperopt==0.2.7
import hyperopt
hyperopt.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'0.2.7'

In [ ]:
import time

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from numpy.random import RandomState

In [ ]:
from hyperopt import hp, fmin, tpe

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

In [ ]:
from hyperopt import hp, fmin, tpe
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

In [ ]:
import hyperopt
hyperopt.__version__

'0.2.7'

其中hp是参数空间创建函数，fmin是参数搜索函数，tpe则是一种基于贝叶斯过程的搜索策略。

## 导入数据

我们使用的数据是压缩之后的最终数据：Part I + Part II + Part III

保证一行对应一个唯一客户。

In [ ]:
train_LGBM = pd.read_parquet("../data/8-CombinedData/combined-data-train.parquet")
test_LGBM = pd.read_parquet("../data/8-CombinedData/combined-data-test.parquet")

In [ ]:
train_LGBM.shape

(458913, 2001)

In [ ]:
test_LGBM.shape

(924621, 2000)

In [ ]:
train_LGBM = train_LGBM.fillna(0)
test_LGBM = test_LGBM.fillna(0)

In [ ]:
test_LGBM

,customer_ID,B_38&5&S_9,D_114&1&S_13,D_114&0&D_79,D_64&-1&D_91,D_117&4&R_4,D_64&0&R_4,B_30&0&R_2,D_114&1&D_111,D_114&1&D_74,...,D_120_last_1,D_64_last_0,D_64_last_2,D_64_last_3,D_68_last_1,D_68_last_2,D_68_last_3,D_68_last_4,D_68_last_5,D_68_last_6
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-0.174560,-0.661302,-0.202459,-4.116865,-0.136289,-0.176305,-0.269203,1.128288,-0.464897,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,-0.103915,0.635275,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,-0.874789,-0.464897,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,-0.174560,-0.661302,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,1.128288,-0.464897,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,-0.174560,-0.661302,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,1.128288,-0.464897,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,-0.116427,-0.661302,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,1.128288,-0.464897,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,-0.174560,-0.661302,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,-0.874789,-0.273857,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.124727,0.027504,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,-0.874789,0.044543,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,-0.174560,1.016144,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,-0.874789,4.788704,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,-0.174560,-0.499230,-0.202459,0.254415,-0.136289,-0.176305,-0.269203,-0.874789,3.260384,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
train_LGBM

,customer_ID,B_38&5&S_9,D_114&1&S_13,D_114&0&D_79,D_64&-1&D_91,D_117&4&R_4,D_64&0&R_4,B_30&0&R_2,D_114&1&D_111,D_114&1&D_74,...,D_64_last_0,D_64_last_2,D_64_last_3,D_68_last_1,D_68_last_2,D_68_last_3,D_68_last_4,D_68_last_5,D_68_last_6,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.219021,2.189610,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.269386,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,-0.219021,-0.024353,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.468583,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.461414,-0.468583,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,-0.219021,1.193602,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.368984,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,0.328206,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,1.145774,-0.468583,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,1.025397,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,-0.219021,1.244000,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.943571,-0.468583,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,-0.219021,-0.633330,-0.190002,0.309325,-0.140324,-0.179559,-0.270268,-0.461414,-0.070188,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1


## Metric

In [ ]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'AMEX_metric', amex_metric(y_true, y_pred), True

## 贝叶斯优化

### 参数回调函数

对于lgb模型来说，并不是所有的超参数都需要进行搜索。

为了防止多次实例化模型过程中部分超参数被设置成默认参数，我们首先需要创建一个参数回调函数，用于在后续多次实例化模型过程中反复申明这部分参数的固定取值：

In [ ]:
def params_append(params):
    """
    动态回调参数函数，params视作字典
    :param params:lgb参数字典
    :return params:修正后的lgb参数字典
    """
    params['feature_pre_filter'] = False
    params['objective'] = "binary"
    params['metric'] = "None" # 将用自定义metric（feval中声明）
    params["is_unbalance"] = True # 不平衡数据
    params["boosting"] = "gbdt" # gbdt, rf, dart
    params["verbose"] = -1
    # params["n_estimators"] = 200
    # params["device"] = "gpu"
    # params["gpu_platform_id"] = 0
    # params["gpu_device_id"] = 0



    return params

### 寻找最优超参数

In [ ]:
lgb.__version__

'3.1.1'

In [ ]:
def param_hyperopt(train):
    """
    模型参数搜索与优化函数
    :param train:训练数据集
    :return params_best:lgb最优参数
    """
    
    # Part 1.划分特征名称，删除ID列和标签列
    label = "target"
    features = train.columns.tolist()
    features.remove("customer_ID")
    features.remove("target")
    
    # Part 2.封装训练数据
    train_data = lgb.Dataset(train[features], train[label])
    
    # Part 3.内部函数，输入模型超参数损失值输出函数
    def hyperopt_objective(params):
        """
        输入超参数，输出对应损失值
        :param params:
        :return:最大自定义metric
        """
        # 创建参数集
        params = params_append(params)
        print(f"检查使用的LGBM参数：{params}")

        t = time.time() # 记录时间
        
        # 借助lgb的cv过程，输出某一组超参数下损失值的最小值
        res = lgb.cv(params, train_data, 
                     num_boost_round = 300,# 最大迭代次数 5000
                     nfold=5, # 交叉验证的次数（n折交叉验证）
                     stratified=True, # 不平衡数据
                     shuffle=True,
                     metrics="None",
                     early_stopping_rounds=100, #dart模式下面不需要early stopping
                     show_stdv=False,
                     seed=2022,
                     verbose_eval=50, # 1000
                     feval = lgb_amex_metric,
                     eval_train_metric=True,
                    )
        
        # 追踪记录
        dur = round((time.time() - t) / 60, 2)
        print(f"本次贝叶斯优化evaluation的消耗时间 {dur} mins")
        
        # 打印训练后的字典
        # print(f"交叉验证后的结果字典：{res}")

        
        return -max(res["valid AMEX_metric-mean"]) # 最大化自定义metric，但请注意我们在贝叶斯优化中的目标函数是最小化，所以要在前面加上负号
    

    # Part 4.lgb超参数空间
    params_space = {
        'learning_rate': hp.uniform('learning_rate', 5e-3, 5e-1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
        'num_leaves': hp.choice('num_leaves', np.arange(30, 200, 10, dtype=int)),
        # 'reg_alpha': hp.randint('reg_alpha', 0, 10),
        # 'reg_lambda': hp.uniform('reg_lambda', 0, 10),
        'bagging_freq': hp.randint('bagging_freq', 3, 13),
        'min_child_samples': hp.choice('min_child_samples', list(range(10, 50, 5))),
        'max_depth': hp.choice('max_depth', np.arange(5, 30, 3, dtype=int)),
    }
    
    # Part 5.TPE超参数搜索
    params_best = fmin( # 注意是最小化这个最优参数
        hyperopt_objective,
        space=params_space,
        algo=tpe.suggest,
        max_evals=10, # 50
        rstate=np.random.default_rng(2022)
    )
    
    # 天坑！必须要用space_eval处理！
    best_params=space_eval(params_space, params_best)
    
    # 返回最佳参数
    return params_best

开始进行贝叶斯优化，求取最佳的超参数

In [ ]:
best_clf = param_hyperopt(train_LGBM)

检查使用的LGBM参数：{'bagging_fraction': 0.7919796956560885, 'bagging_freq': 7, 'feature_fraction': 0.5728155231585855, 'learning_rate': 0.11142176050178165, 'max_depth': 5, 'min_child_samples': 35, 'num_leaves': 70, 'feature_pre_filter': False, 'objective': 'binary', 'metric': 'None', 'is_unbalance': True, 'boosting': 'gbdt', 'verbose': -1}
[50]	cv_agg's train AMEX_metric: 0.79697	cv_agg's valid AMEX_metric: 0.792198
[100]	cv_agg's train AMEX_metric: 0.817983	cv_agg's valid AMEX_metric: 0.806748
[150]	cv_agg's train AMEX_metric: 0.832588	cv_agg's valid AMEX_metric: 0.814903
[200]	cv_agg's train AMEX_metric: 0.842521	cv_agg's valid AMEX_metric: 0.819147
[250]	cv_agg's train AMEX_metric: 0.850292	cv_agg's valid AMEX_metric: 0.821358
[300]	cv_agg's train AMEX_metric: 0.856692	cv_agg's valid AMEX_metric: 0.822901
本次贝叶斯优化evaluation的消耗时间 18.0 mins
检查使用的LGBM参数：{'bagging_fraction': 0.8450629040529597, 'bagging_freq': 10, 'feature_fraction': 0.8949698698614581, 'learning_rate': 0.3856699583843958, 'ma

### 输出最优参数

In [ ]:
best_clf

{'bagging_fraction': 0.9145573834812892,
 'bagging_freq': 10,
 'feature_fraction': 0.8371096595408576,
 'learning_rate': 0.0584746743556553,
 'max_depth': 6,
 'min_child_samples': 3,
 'num_leaves': 12}

**请注意这里有一个天坑！！！** 我们必须在倒数第二行使用`space_eval(params_space, params_best)`

如果不写，则直接输出`best_clf`是一个字典，这个东西看似是最优参数，其实不全是。如果你在搜索空间里设置的是**枚举类型(hp.choice)**，那么返回的是**索引**而不是真实值！

有的时候会看见`num_leaves`为1，简直是匪夷所思。因为我在参数空间中根本没有设置过这样的数字。其实他是指你在参数空间中声明的可能值的那个数组的下标1对应的数字。

因此我必须要利用`space_eval(参数空间, best_clf)`这个函数将返回的字典结果转化为装有最优参数的结果。

In [ ]:
import pickle
with open("best_parameters.pkl", "wb") as tf:
    pickle.dump(best_clf, tf)

这样我们就得到了通过贝叶斯优化得到的最优超参数。

通过这个最优超参数，我们可以在下面重新训练，并且结合交叉验证进行模型预测。

## 训练模型

### 利用交叉验证进行模型预测

<center><img src="https://s2.loli.net/2021/12/08/ALF3cfuSwmB7b8z.png" alt="image-20211208192640281" style="zoom:33%;" />

自定义最优参数

In [ ]:
best_clf = {'bagging_fraction': 0.9145573834812892, 'bagging_freq': 10, 'feature_fraction': 0.8371096595408576, 
            'learning_rate': 0.0584746743556553, 'max_depth': 23, 'min_child_samples': 25, 'num_leaves': 150, 'feature_pre_filter': False, 
            'objective': 'binary', 'metric': 'None', 'is_unbalance': True, 'boosting': 'gbdt', 'verbose': -1}

In [ ]:
def train_predict(train, test, params):
    """
    :param train:
    :param test:
    :param params:
    :return:
    """
    print("*"*50)
    print("LGBM 开始正式训练！")
    print("*"*50)

    # Part 1.选择特征
    label = "target"
    features = train.columns.tolist()
    features.remove("customer_ID")
    features.remove("target")

    print(f"将要使用的LGBM的最优参数：{params}")

    # Part 2.申明固定参数与控制迭代参数
    params = params_append(params)
    ESR = 500
    NBR = 3000 # 10000训练模型可以调高
    VBE = 1000

    # Part 3.创建结果存储容器
    # 测试集预测结果存储器，后保存至本地文件
    prediction_test = 0
    # 验证集的模型表现，作为展示用
    cv_score = []
    # 验证集的预测结果存储器，后保存至本地文件
    prediction_train = pd.Series()

    # Part 3.交叉验证
    iteration = 1
    kf = StratifiedKFold(n_splits=5, random_state=2022, shuffle=True)
    for train_part_index, eval_index in kf.split(train[features], train[label]):
        print("*"*30)
        print(f"开始第{iteration}折的交叉验证！")
        print("*"*30)
        iteration += 1

        t = time.time() # 记录时间

        
        # 训练数据封装
        train_part = lgb.Dataset(
            train[features].loc[train_part_index],
            train[label].loc[train_part_index]
        )
        # 测试数据封装
        eval = lgb.Dataset(
            train[features].loc[eval_index],
            train[label].loc[eval_index]
        )
        # 依据验证集训练模型
        bst = lgb.train(
            params,
            train_part,
            num_boost_round=NBR,
            valid_sets=[train_part, eval],
            valid_names=['train', 'valid'],
            early_stopping_rounds=ESR,
            verbose_eval=VBE,   
            feval = lgb_amex_metric, # 自定义metric
        )

        # 测试集预测结果并纳入prediction_test容器
        prediction_test += bst.predict(test[features])
        # 验证集预测结果并纳入prediction_train容器
        prediction_train = prediction_train.append(
            pd.Series(bst.predict(train[features].loc[eval_index]),index=eval_index))
        # 验证集预测结果
        eval_pre = bst.predict(train[features].loc[eval_index])

        # 计算验证集上得分metric
        score = amex_metric(train[label].loc[eval_index].values, eval_pre)

        # 纳入cv_score容器
        cv_score.append(score)

        # 追踪记录
        dur = round((time.time() - t) / 60, 2)
        print(f"第{iteration}个iteration训练所消耗的时间 {dur} mins")


    print("*"*50)
    print("LGBM 训练结束！开始保存结果！")
    print("*"*50)
        
    # Part 4.打印/输出结果
    # 打印验证集得分与平均得分
    print(f"验证集得分：{cv_score}, 验证集平均分：{sum(cv_score)/5}")
    
    # 将验证集上预测结果写入本地文件
    pd.Series(prediction_train.sort_index().values).to_csv("train_lightgbm.csv", index=False)
    
    # 将测试集上预测结果写入本地文件
    pd.Series(prediction_test/5).to_csv("test_lightgbm.csv", index=False)
    
    # 测试集平均得分作为模型最终预测结果
    test['target'] = prediction_test/5
    
    # 将测试集预测结果写成竞赛要求格式并保存至本地
    test[["customer_ID", 'target']].to_csv("submission_lightgbm.csv", index=False)

    return

In [ ]:
# best_clf = param_hyperopt(train_LGBM)
train_predict(train_LGBM, test_LGBM, best_clf)

**************************************************
LGBM 开始正式训练！
**************************************************
将要使用的LGBM的最优参数：{'bagging_fraction': 0.9145573834812892, 'bagging_freq': 10, 'feature_fraction': 0.8371096595408576, 'learning_rate': 0.0584746743556553, 'max_depth': 23, 'min_child_samples': 25, 'num_leaves': 150, 'feature_pre_filter': False, 'objective': 'binary', 'metric': 'None', 'is_unbalance': True, 'boosting': 'gbdt', 'verbose': -1}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


******************************
开始第1折的交叉验证！
******************************
